# Request from the API

http://127.0.0.1:8000/?format=api

In [36]:
import requests
import pandas as pd
import seaborn as sns
import plotly.express as px

url_p = "http://127.0.0.1:8000/purchase/?format=json" # reading purchases data
purchases_df = pd.read_json(url_p)

url_c = "http://127.0.0.1:8000/clients/?format=json"  # reading clients data
clients_df = pd.read_json(url_c)
clients_df = clients_df.rename(columns={'id':'client'})

df = purchases_df.merge(clients_df.drop_duplicates(subset=['client']), how='left') # merg the two DataFrames

# Changing data types
df['id'] = df['id'].astype(int)
df['price'] = df['price'].astype(float)
df['purchase_date'] = df['purchase_date'].astype('datetime64[ns]')

df.sample(10)

,id,price,purchase_date,payment_methods,client,name
14,15,120.0,2021-01-18,Cash,3,Jennifer
19,20,249.0,2021-04-23,Credit Card,3,Jennifer
13,14,90.0,2021-01-18,Cash,2,Petter
10,11,130.0,2021-03-09,Cash,1,John
36,37,20.0,2021-04-23,Cash,4,Jessica
3,4,80.0,2018-11-13,Cash,4,Jessica
17,18,120.0,2020-08-05,Cash,5,Sergio
0,1,100.0,2018-01-27,Credit Card,1,John
5,6,160.0,2020-04-21,Cash,6,Mauricio
37,38,70.0,2021-04-23,Cash,4,Jessica


In [37]:
def get_sum_from_client_id(df, id, payment_methods=True):
    
    # Return balance from a client

    filter_df = df[df['client']==id]
    filter_df_sum = filter_df[['price','name','payment_methods','purchase_date']]

    if payment_methods == True:
        x = filter_df_sum.groupby(by=['name','payment_methods']).sum().reset_index()
    else:
        x = filter_df_sum.groupby(by=['name']).sum().reset_index()

    return x

In [38]:
df_id = get_sum_from_client_id(df, 3)
df_id

,name,payment_methods,price
0,Jennifer,Cash,280.0
1,Jennifer,Credit Card,483.0


In [41]:
fig = px.box(df, x="name", y="price", color='payment_methods',title='Customers Boxplots')
fig